# <center>Benchmark compression</center>

## <center> Introduction</center>

Ce notebook sert à exécuter des commandes `ffmpeg` afin de tester les différents paramètres et d'en tirer rapidement des métriques pour améliorer la commande de compression utilisée.

In [23]:
file_in = "/data/pipeline/pipeline-tracking/2022_CF_Limoges/before_jeudi_droit/A008_11030611_C007.mov"
format_out = ".mp4"

In [24]:
import timeit
import subprocess
import shlex
from datetime import timedelta
import os
import ffmpeg

f_time = timeit.default_timer

# get the metadata of the input video once and for all
file_duration = float(ffmpeg.probe(file_in)["streams"][0]["duration"])
in_size = float(ffmpeg.probe(file_in)["format"]["size"])

Error: ffprobe error (see stderr output for detail)

## <center>Exécution des commandes</center>

### 3. Sous-échantillonnage colorimétrique

In [ ]:
commands = ["ffmpeg -y -i $in -vcodec libx264 -crf 24 -preset ultrafast -acodec aac -strict experimental $out",                #4:2:2 10-bit
    "ffmpeg -y -i $in -vcodec libx264 -crf 24 -preset ultrafast -vf format=yuv422p -acodec aac -strict experimental $out",     #4:2:2 8-bit
    "ffmpeg -y -i $in -vcodec libx264 -crf 24 -preset ultrafast -vf format=yuv420p10le -acodec aac -strict experimental $out", #4:2:0 10-bit
    "ffmpeg -y -i $in -vcodec libx264 -crf 24 -preset ultrafast -vf format=yuv420p -acodec aac -strict experimental $out",     #4:2:0 8-bit
    "ffmpeg -y -i $in -vcodec libx264 -crf 24 -preset ultrafast -vf format=yuv411p -acodec aac -strict experimental $out",     #4:1:1 8-bit
    
    "ffmpeg -y -i $in -vcodec libx264 -crf 24 -preset superfast -acodec aac -strict experimental $out",                        #4:2:2 10-bit
    "ffmpeg -y -i $in -vcodec libx264 -crf 24 -preset superfast -vf format=yuv422p -acodec aac -strict experimental $out",     #4:2:2 8-bit
    "ffmpeg -y -i $in -vcodec libx264 -crf 24 -preset superfast -vf format=yuv420p10le -acodec aac -strict experimental $out", #4:2:0 10-bit
    "ffmpeg -y -i $in -vcodec libx264 -crf 24 -preset superfast -vf format=yuv420p -acodec aac -strict experimental $out",     #4:2:0 8-bit
    "ffmpeg -y -i $in -vcodec libx264 -crf 24 -preset superfast -vf format=yuv411p -acodec aac -strict experimental $out",     #4:1:1 8-bit   
]

Comp = [0]*len(commands)
Durees = [0]*len(commands)

Exécution du script

In [25]:
for i, command in enumerate(commands):
        file_out = str(i) + format_out
        print("Commande :", command)
        print("Output :", file_out)
        
        c_time0 = f_time()
        open(file_out, 'a').close() # create empty file

        cmd = shlex.split(command.replace("$in", file_in).replace("$out", file_out))
        # renvoie liste ["ffmpeg", "-y", "$in"...]
        # .replace("$in", file_in) renvoie ["ffmpeg", "-y", file_in...]
        
        #ffmpeg renvoie bcp bcp de texte (essayer dans un terminal), on log que
        #les erreurs qui empêcheraient l'exécution correcte du code
        cmd.append("-loglevel"); cmd.append("error")
        
        subprocess.call(cmd) # execute une commande sous forme de liste (cf. au dessus)
        
        c_time1 = f_time()
        print("Durée de compression :", str(timedelta(seconds = (c_time1 - c_time0))))
        print("Par rapport à la durée :", str( (c_time1 - c_time0)/file_duration ))
        Durees[i] = c_time1 - c_time0

        out_size = os.path.getsize(file_out)
        print("Ratio de compression :", str( out_size/in_size ))
        print("")
        Comp[i] = out_size/in_size

Commande : ffmpeg -y -i $in -vcodec libx264 -crf 24 -preset ultrafast -acodec aac -strict experimental $out
Output : 0.mp4
Durée de compression : 0:00:00.042605


Unrecognized option 'crf'.
Error splitting the argument list: Option not found


NameError: name 'file_duration' is not defined